In [1]:
"""This is my 'data acquisition' notebook"""

"This is my 'data acquisition' notebook"

In [95]:
"""Imports"""
import requests
from bs4 import BeautifulSoup
import re

In [200]:
def get_property_data(property_url):
    """This function will receive the url of the property and parse its contents. It will then save the property's data in a dictionary."""
    property_headers = []
    property_data = []
    property = requests.get(property_url).text
    property_soup = BeautifulSoup(property, "html.parser")
    for tr in property_soup.find_all("tr"):
        for th in tr.find_all("th"):
            header = str(th.string)
            header_strip = header.strip()
            property_headers.append(header_strip)
        for td in tr.find_all("td"):
            raw_datum = str(td)
            sub_datum_1 = re.sub(r"<.*?>", '', raw_datum).strip()
            sub_datum_2 = re.sub(r"\n" , '', sub_datum_1).strip()
            sub_datum_3 = re.sub(r"\s{2,}", '', sub_datum_2).strip()
            property_data.append(sub_datum_3)
    property_dictionary = dict(zip(property_headers, property_data))
    print(property_dictionary)
    return (property_dictionary)

In [204]:
##Check
get_property_data("https://www.immoweb.be/en/classified/apartment/for-sale/sint-niklaas/9100/10152126?searchId=633d99213408a")

{'Available as of': 'After signing the deed', 'Construction year': '2019', 'Floor': '1', 'Number of floors': '1', 'Building condition': 'As new', 'Street frontage width': '4 m', 'Number of frontages': '2', 'Covered parking spaces': '1', 'Surroundings type': 'Living area (residential, urban or rural)', 'Living area': '50m²square meters', 'Living room surface': '9m²square meters', 'Kitchen type': 'USA hyper equipped', 'Kitchen surface': '6m²square meters', 'Bedrooms': '1', 'Bedroom 1 surface': '10m²square meters', 'Dressing room': 'No', 'Bathrooms': '1', 'Toilets': '1', 'Office': 'No', 'Professional space': 'No', 'Basement surface': '2m²square meters', 'Armored door': 'No', 'Terrace surface': '4m²square meters', 'Terrace orientation': 'South', 'Caretaker': 'No', 'Elevator': 'Yes', 'Accessible for disabled people': 'Yes', 'Intercom': 'Yes', 'Secure access / alarm': 'No', 'Air conditioning': 'No', 'TV cable': 'Yes', 'Visio phone': 'Yes', 'Jacuzzi': 'No', 'Sauna': 'No', 'Swimming pool': 'No

{'Available as of': 'After signing the deed',
 'Construction year': '2019',
 'Floor': '1',
 'Number of floors': '1',
 'Building condition': 'As new',
 'Street frontage width': '4 m',
 'Number of frontages': '2',
 'Covered parking spaces': '1',
 'Surroundings type': 'Living area (residential, urban or rural)',
 'Living area': '50m²square meters',
 'Living room surface': '9m²square meters',
 'Kitchen type': 'USA hyper equipped',
 'Kitchen surface': '6m²square meters',
 'Bedrooms': '1',
 'Bedroom 1 surface': '10m²square meters',
 'Dressing room': 'No',
 'Bathrooms': '1',
 'Toilets': '1',
 'Office': 'No',
 'Professional space': 'No',
 'Basement surface': '2m²square meters',
 'Armored door': 'No',
 'Terrace surface': '4m²square meters',
 'Terrace orientation': 'South',
 'Caretaker': 'No',
 'Elevator': 'Yes',
 'Accessible for disabled people': 'Yes',
 'Intercom': 'Yes',
 'Secure access / alarm': 'No',
 'Air conditioning': 'No',
 'TV cable': 'Yes',
 'Visio phone': 'Yes',
 'Jacuzzi': 'No',
 'S